In [1]:
from dotenv import load_dotenv # type: ignore
import finnhub # type: ignore
import logging # type: ignore
import os # type: ignore
import pandas as pd # type: ignore
import time # type: ignore

try:
    os.remove('stocks.csv')
except:
    pass

load_dotenv()
API_KEY = os.getenv('FINNHUB_API')
API_KEY2 = os.getenv('FINNHUB2_API')
API_KEY3 = os.getenv('FINNHUB3_API')

finnhub_client = finnhub.Client(api_key=f"{API_KEY}")
alt_finnhub_client = finnhub.Client(api_key=f"{API_KEY2}")
alt2_finnhub_client = finnhub.Client(api_key=f"{API_KEY3}")
clients = [finnhub_client, alt_finnhub_client, alt2_finnhub_client]

SYMBOLS = finnhub_client.stock_symbols('US')
SYMBOLS = sorted([item for item in SYMBOLS if item['type'] == 'Common Stock'], key=lambda x: x['symbol'])
STOCKS = {}

period = 0
max_retries = 3
retry_delay = 1
for count, symbol in enumerate(SYMBOLS):
    retries = 0
    while retries < max_retries:
        try:
            client = clients[count % len(clients)]
            recommendations = client.recommendation_trends(symbol['symbol'])

            if count == 0:
                print(f"LAST UPDATED: {recommendations[period]['period']}")
            if count % 1000 == 0:
                print(f"{count} out of {len(SYMBOLS)} complete")

            STOCKS[symbol['symbol']] = 0
            STOCKS[symbol['symbol']] += (2 * recommendations[period]['strongBuy'])
            STOCKS[symbol['symbol']] += (1 * recommendations[period]['buy'])
            STOCKS[symbol['symbol']] += (-1 * recommendations[period]['sell'])
            STOCKS[symbol['symbol']] += (-2 * recommendations[period]['strongSell'])

            break
        except Exception as error:
            if str(error) == 'list index out of range':
                break
            logging.error(f"Error processing {symbol['symbol']}: {error}")
            retries += 1
            time.sleep(retry_delay)
    time.sleep(1.05)

sorted_stocks = dict(sorted(STOCKS.items(), key=lambda item: item[1], reverse=True))
for symbol, score in list(sorted_stocks.items())[:10]:
    print(f'{symbol}: {score}')
print(len(sorted_stocks))

sorted_stocks = list(sorted_stocks.items())
df = pd.DataFrame(sorted_stocks, columns=['Symbol', 'Score'])
df.to_csv('stocks.csv', index=False)

LAST UPDATED: 2025-01-01
0 out of 17460 complete
1000 out of 17460 complete


ERROR:root:Error processing BALY.T: FinnhubAPIException(status_code: 403): You don't have access to this resource.
ERROR:root:Error processing BALY.T: FinnhubAPIException(status_code: 403): You don't have access to this resource.
ERROR:root:Error processing BALY.T: FinnhubAPIException(status_code: 403): You don't have access to this resource.


2000 out of 17460 complete
3000 out of 17460 complete
4000 out of 17460 complete
5000 out of 17460 complete


ERROR:root:Error processing FDP: HTTPSConnectionPool(host='api.finnhub.io', port=443): Read timed out. (read timeout=10)


6000 out of 17460 complete
7000 out of 17460 complete
8000 out of 17460 complete
9000 out of 17460 complete
10000 out of 17460 complete
11000 out of 17460 complete
12000 out of 17460 complete
13000 out of 17460 complete
14000 out of 17460 complete
15000 out of 17460 complete


ERROR:root:Error processing TCGN: HTTPSConnectionPool(host='api.finnhub.io', port=443): Read timed out. (read timeout=10)


16000 out of 17460 complete
17000 out of 17460 complete
AMZN: 94
META: 88
NVDA: 88
MSFT: 82
TCTZF: 75
GOOG: 72
GOOGL: 72
UBER: 70
AMD: 62
MPNGF: 62
17459
